In [4]:
### 本程序由于深交所放开创业板限制，专门为创业板定制
import requests
import json

tonghuashun_token = '6ca2a3c7e9199b5468c08a1979f1dda5'
tonghuashun_v = 'ArveOVDVYskUsl59L8hOl78CTJQmEM_BySWTqa14l9LTk9UCNeBfYtn0Ix--'

def tonghuashun(tonghuashun_token, v):
    pre = 'http://www.iwencai.com/stockpick/cache?token='
    post = '&p=1&perpage='
    url = pre + tonghuashun_token + post
    headers = {'User-Agent':'Mozilla/5.0 \(Windows NT 6.1; WOW64\) AppleWebKit/537.36 \(KHTML, like Gecko\) Chrome/39.0.2171.71 Safari/537.36'}
    c = 'cid=d1ac40a4d5f4281a1116cb79aadd04d31551921949; ComputerID=d1ac40a4d5f4281a1116cb79aadd04d31551921949; ver_mark=d; PHPSESSID=e7fa610e42fe384ee1501c8ee4d525e3; guideState=1'
    cookies = {}
    aa = c.split(';')
    for line in aa:
        key, value = line.split('=', 1)
        cookies[key] = value
    cookies['v'] = v    
    response = requests.get(url + '10', headers = headers, cookies = cookies)
    totalStr = response.text.split('"total":')[-1].split(",")[0]
    final_url = url + totalStr
    response = requests.get(final_url, headers = headers, cookies = cookies)
    wccode2hq = response.text.split('"wccode2hq":')[-1].split(',"orresProcess"')[0]
    wccode2hq_dict = json.loads(wccode2hq)
    tonghuashun_codes = list(wccode2hq_dict.keys())
    codes = [code.split('.')[0] for code in tonghuashun_codes]
    return codes
aa = tonghuashun(tonghuashun_token, tonghuashun_v)
print(len(aa))


82


In [5]:
print(aa)

['300022', '300023', '300025', '300029', '300032', '300040', '300042', '300051', '300062', '300069', '300071', '300076', '300080', '300090', '300095', '300097', '300111', '300129', '300165', '300167', '300169', '300175', '300211', '300239', '300240', '300249', '300260', '300264', '300277', '300279', '300281', '300293', '300306', '300312', '300313', '300330', '300354', '300356', '300362', '300380', '300391', '300392', '300405', '300407', '300421', '300431', '300453', '300460', '300461', '300475', '300479', '300480', '300488', '300489', '300492', '300507', '300508', '300514', '300526', '300536', '300538', '300539', '300540', '300547', '300549', '300552', '300555', '300566', '300576', '300582', '300587', '300593', '300598', '300626', '300635', '300648', '300656', '300667', '300671', '300698', '300710', '300713']


In [6]:
import pandas as pd
import json
from pandas import DataFrame

## 按题材加权积分 ##
level0 = ['汽车整车', '房地产', '保险', '银行', '港口航运', '化工材料', 
          '家用电器', '白酒', '华为电视', '华为汽车', '超高清视频', '种植业', '大飞机', 
          '国产航母', '高校', '旅游', '国企改革', '文化传媒', '养老概念',
          '边缘计算', '地下管网', '健康中国', '工业4.0', '降税']
level1 = ['石墨烯', '业绩增长', '海南', '摘帽', '操作系统', '国产软件',
          '成渝特区', '华为海思', '华为概念','送转填权', '无人驾驶', '物流', '农药',
          '广电网', '雄安新区', '透明工厂', '新能源车', '页岩气', '光伏', '风电', '有色金属', 
          '电力物联网', 'OLED', '创投', '一带一路', '养殖业', '军工', '贬值收益', '大数据', 
          '振兴东北', '贸易战概念', '乡村振兴', '基础建设']
level2 = ['稀土永磁', '5G', '人造肉', '高送转', '大麻', '氢燃料电池', '芯片', 
          '云计算', '区块链', '黄金', '网络安全', '核电', 
          '次新股', '游戏', '医药', '低价', '知识产权', '并购重组']
level3 = ['壳资源', '环保', '金融科技', '证券', '石油石化', '超跌'] 

# print(len(level0))
# print(len(level1))
# print(len(level2))
# un = list(set(level0).union(set(level1)).union(set(level2)))
# print(len(un))

df = pd.read_csv('/data/stock_new/topics' + '.csv', header=None, names = ['code', 'name', 'topics'])

pd.set_option('display.width', 1000)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行

result = DataFrame([], columns=['code', 'name', 'score', 'topics'])
failedCodes = []
for index, row in df.iterrows():
    try:
        if index % 200 == 0:
            print(str(index) + '个已处理...')
        score = 0
        topics = row.topics.split('[\'')[-1].split('\']')[0].split('\', \'')
        for topic in topics:
            if topic in level1:
                score = score + 1
            elif topic in level2:
                score = score + 2
            elif topic in level3:
                score = score + 3
        result = result.append({'code':row.code, 'name':row.iloc[1], 'score':score, 'topics':row.topics}, ignore_index = True)
    except BaseException as e:
        failedCodes.append(row.code)
        pass
    continue
final_result = result.sort_values(by="score" , ascending=False) 
print(final_result)

0个已处理...
200个已处理...
400个已处理...
600个已处理...
800个已处理...
1000个已处理...
1200个已处理...
1400个已处理...
1600个已处理...
1800个已处理...
2000个已处理...
2200个已处理...
2400个已处理...
2600个已处理...
2800个已处理...
3000个已处理...
3200个已处理...
3400个已处理...
3600个已处理...
           code   name score                                             topics
50    000066.SZ   中国长城    15  ['深圳', '光伏概念', '电子信息', '操作系统', '军工', '并购重组', '...
1941  300561.SZ   汇金科技    14  ['计算机应用', '广东其它', '金融科技', '物联网', '操作系统', '高送转'...
1538  300152.SZ   科融环境    13  ['脱硫脱硝', 'PM2.5', '长三角', '环保', '超跌', '低价', '江苏...
471   002012.SZ   凯恩股份    13  ['互联网金融', '壳资源', '氟化工', '军工', '并购重组', '氢燃料电池',...
920   002464.SZ   众应互联    13  ['电子商务', '高送转', '股权转让', '江苏', '通信服务', '网络游戏', ...
205   000633.SZ   合金投资    12  ['壳资源', '核电', '一带一路', '民营医院', '超跌', '健康中国', '新...
2038  300663.SZ   科蓝软件    12  ['计算机应用', '北京', '数字中国', '互联网金融', '无人零售', '金融科技...
2448  600355.SH   精伦电子    12  ['壳资源', '电子信息', '物联网', '湖北', 'OLED', '低价', '云计...
958   002502.SZ   骅威文化    11  ['广东其它', '文化传媒', '传媒', '二胎概念'

In [10]:
from pandas import DataFrame

def getCodeWithSuffix(code):
    if code[0] == '6':
        return code + '.SH'
    else: 
        return code + '.SZ'


codes1 = []
for a in aa:
    codes1.append(getCodeWithSuffix(a))
print(codes1)

bbb = DataFrame([], columns=['code', 'name', 'score', 'topics'])
for index,row in final_result.iterrows():
    if row.code in codes1:
        bbb = bbb.append(row)
print(bbb)

['300022.SZ', '300023.SZ', '300025.SZ', '300029.SZ', '300032.SZ', '300040.SZ', '300042.SZ', '300051.SZ', '300062.SZ', '300069.SZ', '300071.SZ', '300076.SZ', '300080.SZ', '300090.SZ', '300095.SZ', '300097.SZ', '300111.SZ', '300129.SZ', '300165.SZ', '300167.SZ', '300169.SZ', '300175.SZ', '300211.SZ', '300239.SZ', '300240.SZ', '300249.SZ', '300260.SZ', '300264.SZ', '300277.SZ', '300279.SZ', '300281.SZ', '300293.SZ', '300306.SZ', '300312.SZ', '300313.SZ', '300330.SZ', '300354.SZ', '300356.SZ', '300362.SZ', '300380.SZ', '300391.SZ', '300392.SZ', '300405.SZ', '300407.SZ', '300421.SZ', '300431.SZ', '300453.SZ', '300460.SZ', '300461.SZ', '300475.SZ', '300479.SZ', '300480.SZ', '300488.SZ', '300489.SZ', '300492.SZ', '300507.SZ', '300508.SZ', '300514.SZ', '300526.SZ', '300536.SZ', '300538.SZ', '300539.SZ', '300540.SZ', '300547.SZ', '300549.SZ', '300552.SZ', '300555.SZ', '300566.SZ', '300576.SZ', '300582.SZ', '300587.SZ', '300593.SZ', '300598.SZ', '300626.SZ', '300635.SZ', '300648.SZ', '300656.SZ'

In [12]:
with pd.ExcelWriter('创业板题材积分排名.xlsx') as writer:
    bbb.to_excel(writer, sheet_name='开盘啦数据')
print('success')

success
